In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import os
import random
import numpy as np
import copy

In [ ]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True, warn_only=True)

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)


In [ ]:
!wget https://www.dropbox.com/scl/fi/w22pt8h447b9ptgql67vo/dataset.tar.gz?rlkey=vajo7g4w8nl1q92ikv8qu75qu&dl=0

In [ ]:
!mv dataset.tar.gz?rlkey=vajo7g4w8nl1q92ikv8qu75qu dataset.tar.gz

In [ ]:
!tar -xzvf dataset.tar.gz

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print("device is {}".format(device))

In [ ]:
def read_image_tensor(image_folder,transform,num_images=None):
    if num_images==None:
        num_images = len(os.listdir(image_folder))
    images = []
    for i in range(num_images):
        img = torchvision.io.read_image(os.path.join(image_folder,f"{i}.jpg")).float()
        images.append(transform(img))
    return torch.stack(images).to(device)

In [ ]:
def get_labels(csv_file):
    # TODO: Copy this from the Colab notebook in Q1

In [ ]:
img_size = (256,256)
base_transform = transforms.Compose(
    [transforms.Resize(img_size)
    ]
)
train_X = read_image_tensor("dataset/train/",base_transform)/256
train_Y = get_labels("dataset/train.csv")
valid_X = read_image_tensor("dataset/test/",base_transform)/256
valid_Y = get_labels("dataset/test.csv")

In [ ]:
train_dataset = TensorDataset(train_X, train_Y)
valid_dataset = TensorDataset(valid_X, valid_Y)
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# you can use this utility function to get the number of trainable parameters in your model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
set_seed(42)
# TODO: Load a pretrained model VGG-11 into baseModel
# Refer to https://pytorch.org/docs/stable/hub.html#torch.hub.load on how to load a pretrained vgg11 model
# VGG model: https://pytorch.org/hub/pytorch_vision_vgg/
baseModel = None
baseModel = #TODO: Complete definition

# TODO: Freeze all the params of the VGG-11 model
# Make sure that gradients are not backpropagated through the VGG-11 model

# Once frozen correctly, the following statement should print that the number of trainable params is 0
print("Number of trainable params in base model is ", count_parameters(baseModel))

In [ ]:
# Building the classifier which will use the pretrained VGG-11 model's features as input
class Classifier(nn.Module):

  def __init__(self, baseModel, numOutputNeurons):
    super().__init__()
	  # initialize the base model

    self.baseModel = baseModel
    self.baseModel.classifier =
      #TODO: Complete this definition
      # Add a linear layer to project down to 1024 nodes, followed by ReLU, a dropout layer with p = 0.5 and another linear layer
      # projecting the 1024 nodes down to numOutputNeurons (= 1 in our problem)
      # Finally, a sigmoid layer is added for the output probabilities

    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    # TODO: Complete forward definition
    # TODO: return the classifier's output probabilities saved in probs
    return probs


In [ ]:
device = torch.device(device)
model = None
model = Classifier(baseModel=baseModel, numOutputNeurons=1)
model = model.to(device)

print("Model is ")
print(model)
print("Trainable params of new model with classifier head is ", count_parameters(model))

In [ ]:
# initialize loss function and optimizer
num_epochs = 30
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train_model(model, train_loader, valid_loader, num_epochs, loss_function, optimizer):
    # TODO: Note how the best checkpoint is saved based on validation accuracy

    set_seed(42)
    prev_acc = 0.0
    best_checkpoint = None

    for epoch in range(num_epochs):
        model.train()

        total_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            optimizer.zero_grad()
            output = model(inputs)

            loss = loss_function(output, labels.view(output.shape))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss/len(train_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")
        model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data
                outputs = model(inputs)
                pred = (outputs > 0.5)*1
                correct += (pred==labels.view(pred.shape)).sum()
                total += labels.size(0)
            accur = 100*correct/total
            print(f"Test Accuracy after Epoch {epoch+1}: {accur:.2f}%")
            if accur > prev_acc:
              print("Saving best checkpoint")
              prev_acc = accur
              best_checkpoint = copy.deepcopy(model)


    print("Training done.")
    return best_checkpoint

In [ ]:
best_checkpoint = train_model(model, train_loader, valid_loader, num_epochs, loss_func, optimizer)

In [ ]:
def evaluate(model, test_loader):

  set_seed(42)
  model.eval()
  predictions = []

  # TODO: Evaluate model and generate binary outputs for all the test instances
  # in test_loader. Return the predicted outputs in a list named predictions.
  return predictions


## Evaluate using best checkpoint on Kaggle test set

In [ ]:
img_size = (256,256)
base_transform = transforms.Compose(
    [transforms.Resize(img_size)
    ]
)
kaggle_X = read_image_tensor("dataset/kaggle/",base_transform)/256
kaggle_dataset = TensorDataset(kaggle_X)
batch_size = 64

kaggle_loader = DataLoader(kaggle_dataset, batch_size=batch_size, shuffle=False)
kaggle_predictions = evaluate(best_checkpoint, kaggle_loader)

ids = [i for i in range(len(kaggle_predictions))]
pred_dict = {"id": ids, "label": kaggle_predictions}
df = pd.DataFrame(pred_dict)
df.to_csv("./submission.csv", index=False)



## Training the last few layers of vgg-11 along with the classifier

In [ ]:
set_seed(42)
baseModel = None
# TODO: Copy from an earlier cell where baseModel is initialized to a pretrained VGG-11

In [ ]:
print(count_parameters(baseModel))

# TODO: Except for parameters in the last layer or two, freeze the rest.
# The two print statements will show the initial number of trainable parameters
# in baseModel and the substantially smaller (almost by a factor of 100)
# number of trainable parameters after implementing the TODO.

print(count_parameters(baseModel))

In [ ]:
device = torch.device(device)
model = None
model = Classifier(baseModel=baseModel, numOutputNeurons=1)
model = model.to(device)

print("Model is ")
print(model)
print("Trainable params of new model with classifier head is ", count_parameters(model))

In [ ]:
# initialize loss function and optimizer
num_epochs = 30
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
best_checkpoint_ft = train_model(model, train_loader, valid_loader, num_epochs, loss_func, optimizer)

In [ ]:
batch_size=64
kaggle_loader = DataLoader(kaggle_dataset, batch_size=batch_size, shuffle=False)
kaggle_predictions = evaluate(best_checkpoint_ft, kaggle_loader)

ids = [i for i in range(len(kaggle_predictions))]
pred_dict = {"id": ids, "label": kaggle_predictions}
df = pd.DataFrame(pred_dict)
df.to_csv("./submission.csv", index=False)

# Submit submission.csv to Kaggle